---

## Set Up System Tools

[Homebrew](http://brew.sh/) installs the stuff you need that Apple didn’t. From a command line (denoted by `$`), enter the following:

```
$ /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
```

Insert the Homebrew directory at the top of your `PATH` environment variable by adding the following to your `~/.profile` (or `~/.zshrc`) file:

```
export PATH=/usr/local/bin:/usr/local/sbin:$PATH
```

Also install `git`, for version control and (later) access to the app's code:

```
$ brew install git
```

---

## Set Up PostgreSQL

Install with Homebrew:

```
$ brew update && brew install postgres
```

Or, if already installed,

```
$ brew update && brew upgrade postgres
```

Run `initdb` just once, basically to create the directory structure and such on disk that's needed for creating new databases. Note: The specified path should match the version of Postgres just installed!

```
$ initdb /usr/local/var/postgres9.5 -E utf8
```

To _manually_ start and stop a local Postgres server from running, use

```
$ pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start
$ pg_ctl -D /usr/local/var/postgres stop -s -m fast
```
        
Or to _automatically_ start a Postgres server (now and) at launch:

```
$ mkdir -p ~/Library/LaunchAgents
$ ln -sfv /usr/local/opt/postgresql/*.plist ~/Library/LaunchAgents
$ launchctl load ~/Library/LaunchAgents/homebrew.mxcl.postgresql.plist
```

Open the system paths file, `/etc/paths`, in a text editor, and move the line `/usr/local/bin` from the bottom of the file to the top (if it wasn't like this already). If you had to make a change to the file, reboot the computer. After rebooting, the command `which psql` command should return `/usr/local/bin/psql`.

Homebrew automatically created a database superuser account with the same login as your current Mac OS account. Let's create a dedicated user named `app` for connecting to and owning the app's database:

```
$ createuser --echo --pwprompt --superuser --createdb app
```

You'll be prompted to create a password — be sure to remember it or save it somewhere!

Homebrew also automatically created a database named `postgres` that may be used to log info for administrative tasks such as creating a user. Let's create a new database for this project:

```
$ createdb --echo --encoding=utf8 --host=<HOST> --port=<PORT> --username=app --owner=app colandr
```

We're using our `app` user to create the database, and assign it as the db's owner.

To access the database through an interactive shell:

```
$ psql --host=<HOST> --port=<PORT> --username=app --dbname=colandr
```

While we're in the interactive shell, let's create the `pgcrypto` extension for secure password storage and `intarray` extension for integer array handling:

```
=# CREATE EXTENSION "pgcrypto";
=# CREATE EXTENSION "intarray";
```

Lastly, after exiting the postgres shell, define a `DATABASE_URL` environment variable containing this same information so that programs such as those in `cipy` can access it:

```
$ export DATABASE_URL=postgres://app:<PASSWORD>@<HOST>:<PORT>/colandr
```

**Update:** Define this environment variable instead:

```
$ export SQLALCHEMY_DATABASE_URI="postgresql://app:{DB_PASS}@{DB_HOST}:{DB_PORT}/colandr"
```

---

## Set Up the App Code

First you'll need access permissions to the repo — contact DataKind! Then, "clone" the code needed to run the app from its repository hosted on GitLab. Make a new local directory for the repo (if needed) and change your current working directory to it:

```
$ mkdir /path/to/cloned_repo
$ cd /path/to/cloned_repo
$ git clone http://gitlab.datakind.org/conservation-intl/conservation-intl.git
```

This command should create a `conservation-intl` directory, in which the app's code lives.

---

## Set Up Python 3

For a fuller guide to everything below, check out [The Hitchhiker's Guide to Python](http://docs.python-guide.org/en/latest/).

Install Python 3 with Homebrew:

```
$ brew install python3
```

You may wish to work entirely within a virtual environment. (See [here](http://docs.python-guide.org/en/latest/dev/virtualenvs/?highlight=virtualenv) for more information on working within virtual envs.) If so, install with `pip`:

```
$ pip3 install virtualenv
```

You may wish to develop within an IPython interpreter and/or Jupyter Notebook:

```
$ pip3 install ipython jupyter
```

Change your working directory to your local copy of the repo (see the section above), then install all 3rd-party dependencies upon which the app will rely by:

```
$ cd /path/to/conservation-intl
$ pip install -r requirements.txt
```

Also add the repo to your `PYTHONPATH` environment variable by modifying the corresponding line (or adding a new line) in your `~/.profile` (or `~/.zshrc`) file:

```
export PYTHONPATH=/path/to/conservation-intl/:$PYTHONPATH
```

---

## Initialize, Migrate, Upgrade, and Reset Database Tables

To initialize a brand new database, navigate to the `conservation-intl/ciapi` directory (within the cloned repo). If the `ciapi/migrations` directory does not exist (note: it _should_), issue the following command:

```
$ python migrate.py db init
```

This initializes the directory and its contents. If database models have changed _or_ a migration script does not exist in the repo (again: it _should_), you'll have to generate an initial migration script using Alembic:

```
$ python migrate.py db migrate
```

The migration script needs to be reviewed and edited, as Alembic doesn't necessarily detect every change you make to models. There's also a known bug when dealing with `postgresql` ARRAYs (see [here](https://bitbucket.org/zzzeek/alembic/issues/85/using-postgresqlarray-unicode-breaks)) that directly affects us. Open the newly-created `ciapi/migrations/versions/<version>.py` module, then do a find-and-replace to convert every instance of "`ARRAY(Unicode`" with "`ARRAY(sa.Unicode`". If you skip this, the next command will throw an exception pointing you to a problematic line. Be sure to add the migration script (and the whole directory) to version control!

Lastly, apply the migration to the database:

```
$ python migrate.py db upgrade
```

Any time database models change, the `db migrate` and `db upgrade` commands have to be run again. To sync the database in another system, just update the `ciapi/migrations` directory from version control (e.g. `git pull`) and run the `db upgrade` command only.

In order to "reset" the database by dropping and then re-creating all of its tables, run the following script:

```
$ python reset_db.py
```

---

**DEPRECATED!**

## Create Tables and Indexes from DDLs

In [1]:
# import logging

# import cipy

In [2]:
# logger = logging.getLogger('cipy')
# logger.setLevel(logging.DEBUG)

In [3]:
# conn_creds = cipy.db.get_conn_creds(env_var='DATABASE_URL')

# for ddl_name in cipy.DEFAULT_DDL_PATHS:
#     ddl = cipy.db.get_ddl(ddl_name)
#     print('\n{}'.format(ddl))
    
#     pgdb = cipy.db.PostgresDB(conn_creds, ddl=ddl)
#     pgdb.create_table(act=True)
#     print(ddl.create_table_statement())
    
#     pgdb.create_indexes(act=True)
#     for index_statement in ddl.create_index_statements():
#         print(index_statement)